## Sentiment Analysis

In [7]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from tqdm import tqdm

In [8]:
# Load dữ liệu đã tiền xử lý
df = pd.read_csv("data/2025-10-31/playlist_data_clean.csv")

In [9]:
from transformers import pipeline

sentiment_model = pipeline(
    "sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment-latest"
)

df["sentiment"] = df["combined_text"].apply(
    lambda x: sentiment_model(x[:512])[0]['label'] if isinstance(x, str) and len(x) > 0 else "neutral"
)

print(df["sentiment"].value_counts())


pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cpu


model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

sentiment
neutral     48
positive     1
Name: count, dtype: int64


In [10]:
df.to_csv("data/2025-10-31/playlist_data_sentiment.csv", index=False, encoding="utf-8")
print("✅ Đã lưu kết quả phân tích cảm xúc!")

✅ Đã lưu kết quả phân tích cảm xúc!


In [11]:
df_clone = df.copy()

In [12]:
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

def get_sentiment(text):
    if not isinstance(text, str) or text.strip() == "":
        return "neutral"
    score = sia.polarity_scores(text)["compound"]
    if score > 0.05:
        return "positive"
    elif score < -0.05:
        return "negative"
    else:
        return "neutral"

df_clone["sentiment"] = df_clone["combined_text"].apply(get_sentiment)
print(df_clone["sentiment"].value_counts())


LookupError: 
**********************************************************************
  Resource [93mvader_lexicon[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('vader_lexicon')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93msentiment/vader_lexicon.zip/vader_lexicon/vader_lexicon.txt[0m

  Searched in:
    - 'C:\\Users\\quocla/nltk_data'
    - 'c:\\Users\\quocla\\AppData\\Local\\Programs\\Python\\Python310\\nltk_data'
    - 'c:\\Users\\quocla\\AppData\\Local\\Programs\\Python\\Python310\\share\\nltk_data'
    - 'c:\\Users\\quocla\\AppData\\Local\\Programs\\Python\\Python310\\lib\\nltk_data'
    - 'C:\\Users\\quocla\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - ''
**********************************************************************


## Classification

In [13]:

import pandas as pd

df = pd.read_csv("data/2025-10-31/playlist_data_clean.csv")


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

vectorizer = TfidfVectorizer(
    max_features=5000,
    stop_words='english'
)
X = vectorizer.fit_transform(df["combined_text"])

# Giả sử chia thành 2 cụm: 0 = nội dung tự nhiên, 1 = quảng cáo
kmeans = KMeans(n_clusters=2, random_state=42)
df["cluster"] = kmeans.fit_predict(X)


In [15]:
terms = vectorizer.get_feature_names_out()
order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]

for i in range(2):
    print(f"\n🔹 Cụm {i}:")
    top_terms = [terms[ind] for ind in order_centroids[i, :15]]
    print("Top keywords:", top_terms)



🔹 Cụm 0:
Top keywords: ['cân', 'giảm', 'nioslim', 'quảng', 'phim', 'cáo', 'tỳ', 'vị', 'mộc', 'thúc', 'mỡ', 'mc', 'hà', 'kẹo', 'kera']

🔹 Cụm 1:
Top keywords: ['bạn', 'có', 'thuốc', 'và', 'tăng', 'không', 'uống', 'cho', 'mua', 'giá', 'linh', 'được', 'hộp', 'bao', 'mình']


In [16]:
def label_cluster(row):
    if row["cluster"] == 0:
        return "ad_real"
    else:
        return "ad_fake"
df["ad_type"] = df.apply(label_cluster, axis=1)


In [19]:
print(df["ad_type"].value_counts())

ad_type
ad_fake    40
ad_real     9
Name: count, dtype: int64


In [17]:

df.to_csv("data/2025-10-31/playlist_data_clustered.csv", index=False, encoding="utf-8")
print("✅ Đã lưu kết quả phân cụm nội dung!")

✅ Đã lưu kết quả phân cụm nội dung!


## Merge 2 model

In [18]:
output_path2 = "data/2025-10-31/playlist_data_analyzed.csv"
df.to_csv(output_path2, index=False, encoding="utf-8")
print(f"💾 Đã lưu kết quả phân tích: {output_path2}")

💾 Đã lưu kết quả phân tích: data/2025-10-31/playlist_data_analyzed.csv
